In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyEDM as edm
from PyIF import te_compute as te

/home/paval/papers/CORREDORAS/pablo_python/.conda/lib/python3.10/site-packages/numba/core/decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [2]:
# Paramecium and Didinium (empirical predator prey)
# data from https://doi.org/10.2307/4195

def Paramecium_Didinium(CC = '0.375', **kwargs):
    kwargs.setdefault('data')
    # read from predator_prey_data/CC0.375.txt
    data = pd.read_csv(f'predator_prey_data/CC{CC}.txt', sep = ',')
    return data['t'].values, data['x'].values, data['y'].values

In [3]:
def chaotic_system(corr_yx, corr_xy=0, timesteps = 1000, dt = 1, noise = 0):
    x = np.array([.5])
    y = np.array([.5])
    t = np.arange(0, timesteps, dt)
    for i in range(1, timesteps):
        xtp1 = x[i-1]*(3.8-3.8*x[i-1] - corr_xy*y[i-1]) + np.random.normal(0, noise)
        ytp1 = y[i-1]*(3.5-3.5*y[i-1] - corr_yx*x[i-1]) + np.random.normal(0, noise)
        x = np.append(x, xtp1)
        y = np.append(y, ytp1)
    return t, x, y

In [4]:
def CAM(corr_yx = 0, timesteps = 1000, dt = 1, noise = 0.3):
    x = np.array([0.5])
    y = np.array([0.5])
    t = np.arange(0, timesteps, dt)
    for i in range(1, timesteps):
        xtp1 = 0.5*x[i-1] + 0.2*y[i-1] + np.random.normal(0, noise)
        ytp1 = corr_yx*x[i-1] + 0.7*y[i-1] + np.random.normal(0, noise)
        x = np.append(x, xtp1)
        y = np.append(y, ytp1)
    return t, x, y

In [5]:
def method_CCM(x, y, **kwargs):
    pair = pd.DataFrame({'x': x, 'y': y})

    dfE1 = edm.EmbedDimension(dataFrame=pair, lib = [1, int(len(pair)/2)], pred = [int(len(pair)/2), len(pair)],
                                columns='x', target='x', showPlot=False, maxE=6, tau = 1)
    dfE2 = edm.EmbedDimension(dataFrame=pair, lib = [1, int(len(pair)/2)], pred = [int(len(pair)/2), len(pair)],
                                columns='y', target='y', showPlot=False, maxE=6, tau = 1)
    dfE = pd.concat([dfE1, dfE2])
    E = int(dfE.iloc[dfE['rho'].idxmax()]['E'])
    ccm = edm.CCM(dataFrame=pair, E=E, tau=1, columns='x', target='y', libSizes=[len(pair)], sample=1, showPlot=False)

    return ccm['y:x'].iloc[-1]

In [6]:
def method_TE(x, y, **kwargs):
    k = kwargs.get('k', 1)
    embedding = kwargs.get('embedding', 1)
    return te.te_compute(y, x, safetyCheck=False, k=k, embedding=embedding)

In [8]:
def obtain_pvalues(t, x, y, causal_method, nshuff = 100,  **kwargs): # arguments for the causal method
    
    corr = causal_method(x, y, **kwargs)

    lfrom = np.random.randint(low=0, high=len(x), size=nshuff)
    pval1 = 0
    pval2 = 0
    for i in range(nshuff):
        x1 = np.random.permutation(x)
        x2 = np.concatenate((x[lfrom[i]:], x[:lfrom[i]]))

        corr1 = causal_method(x1, y, **kwargs)
        corr2 = causal_method(x2, y, **kwargs)

        if corr1 > corr:
            pval1 = pval1 + 1
        if corr2 > corr:
            pval2 = pval2 + 1
    
    pval1 = pval1/nshuff
    pval2 = pval2/nshuff

    return corr, pval1, pval2

In [9]:
t, x, y = chaotic_system(corr_yx = 0.4, timesteps = 1000, dt = 1, noise = 0.01)
# causality from x to y
pvalues_TE_chaotic_system =  obtain_pvalues(t, x, y, method_TE, k = 1, embedding = 1, nshuff = 100)
pvalues_CCM_chaotic_system = obtain_pvalues(t, x, y, method_CCM, nshuff = 100)
# now from y to x where there is no causality
pvalues_TE_chaotic_system_switched =  obtain_pvalues(t, y, x, method_TE, k = 2, embedding = 1, nshuff = 100)
pvalues_CCM_chaotic_system_switched = obtain_pvalues(t, y, x, method_CCM, nshuff = 100)

In [156]:
# now from y to x where there is no causality
pvalues_TE_chaotic_system_switched =  obtain_pvalues(t, y, x, method_TE, k = 2, embedding = 1, nshuff = 100)
pvalues_CCM_chaotic_system_switched = obtain_pvalues(t, y, x, method_CCM, nshuff = 100)

In [152]:
pvalues_TE_chaotic_system

(1.429782942105215, 0.0, 0.0)

In [153]:
pvalues_CCM_chaotic_system

(0.97949, 0.0, 0.0)

In [157]:
pvalues_TE_chaotic_system_switched

(-0.7176371035054641, 0.02, 0.0)

In [158]:
pvalues_CCM_chaotic_system_switched

(0.25585, 0.0, 0.03)